In [6]:
from PIL import ImageGrab
import cv2
import numpy as np  

ss_region = (0+500, 360+150, 1920-500, 1440-200)

prev_white = 0
while True:
    ss_img = ImageGrab.grab(ss_region)
    #convert ss_img to numpy array
    ss_img = np.array(ss_img)
    #convert numpy array to opencv image
    img = cv2.cvtColor(ss_img, cv2.COLOR_BGR2RGB)

    #convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    #find white pixels
    curr_white = cv2.inRange(gray, 255, 255)
    #mask using previous white pixels
    white_pixels = cv2.bitwise_and(curr_white, prev_white)

    #find the coordinate of the highest white pixel
    ymax, xmax = np.unravel_index(white_pixels.argmax(), white_pixels.shape)

    if xmax > 20:
        #white pixels left of xmax
        mirr = white_pixels[:, :xmax]
        #mirror image left right
        mirr = mirr[:, ::-1]
        
        #image of zeros of same size as white pixels
        black_img = np.zeros(white_pixels.shape, dtype=np.uint8)
        #concatenate mirr and black_img
        mirr = np.concatenate((mirr, black_img), axis=1)
        #crop mirr
        mirr = mirr[:, :white_pixels.shape[1]-xmax]
        #crop white pixels
        white_pixels = white_pixels[:, :xmax]
        #concatenate white pixels and mirr
        white_pixels = np.concatenate((white_pixels, mirr), axis=1)
        #convert to color
        white_pixels = cv2.cvtColor(white_pixels, cv2.COLOR_GRAY2RGB)
        #add white pixels to image
        img = cv2.add(img, white_pixels)


    prev_white = curr_white
    cv2.imshow("img", img)
    cv2.waitKey(5)


KeyboardInterrupt: 